In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import math
from bayes_opt import BayesianOptimization
import random
import sklearn
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [2]:


# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
desharnais=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\desharnais_dataset.csv",header=None,index_col=0)
columns_desharnais=['TeamExp','ManagerExp','YearEnd','Length','Effort','Transactions','Entities','PointsAdjust','Envergure','PointsNonAjust','Language']
desharnais.set_axis(columns_desharnais,axis='columns',inplace=True)
desharnais.rename_axis("Features", axis=1,inplace=True)
desharnais.rename_axis("Projects", axis=0,inplace=True)
dataset = desharnais

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort']).values  # Features
y = dataset['Effort'].values  # Target (effort)

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.function(reduce_retracing=True)


# Step 3: Feature Selection using Harmony Search
def objective_function(subset):
    selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), subset) if is_selected]
    if len(selected_features) == 0:
        return float('-inf')  # Penalize subsets with no selected features
    
    
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    cross_val_rmse = []
    
    X_selected = X[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    error_list=[]
    
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test = y[train_index], y[val_index]
    
        # Step 3: Build the ANN model.
        model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

        # Step 4: Compile the model.
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Step 5: Train the model.
        model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=0)

        # Step 6: Evaluate the model.
        y_pred = model.predict(X_test)
    
        error = np.mean(np.abs(y_pred - y_test))
        error_list.append(error)
        
    return 1 / (1 + np.mean(error_list))  # Fitness is the inverse of the error (higher is better)
#====================================================================================================================

pbounds_HS = {
    'hms' : (10,30),
    'iterations': (30,100),
    'harmony_rate' : (0.1,0.9)
}

# Harmony Search Parameters
def harmony_search(hms,iterations, harmony_rate):
    num_features = X.shape[1]
    hms = int(hms)  # Harmony memory size
    iterations = int(iterations)  # Number of iterations

    # Initialize harmony memory
    harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

    # Harmony Search Algorithm
    for _ in range(iterations):
        rnd_choice=np.random.choice(hms)
        new_harmony = np.copy(harmony_memory[rnd_choice])
        for i in range(num_features):
            if np.random.rand() < harmony_rate:  # Adjust this probability based on your problem
                new_harmony[i] = 1 - new_harmony[i]
        current_obj = objective_function(harmony_memory[rnd_choice])
        new_obj = objective_function(new_harmony)
        if new_obj > current_obj:
            harmony_memory[rnd_choice] = new_harmony
    return max([objective_function(i) for i in harmony_memory])

optimizer_HS = BayesianOptimization(
        f=harmony_search,
        pbounds=pbounds_HS,
        random_state=42,
        verbose=1,
    )

optimizer_HS.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.
# Print the best hyperparameters found.
best_params_HS = optimizer_HS.max['params']

num_features = X.shape[1]
hms = int(best_params_HS['hms'])  # Harmony memory size
iterations = int(best_params_HS['iterations'])  # Number of iterations
harmony_rate= best_params_HS['harmony_rate']

# Initialize harmony memory
harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

# Harmony Search Algorithm
for _ in range(iterations):
    rnd_choice=np.random.choice(hms)
    new_harmony = np.copy(harmony_memory[rnd_choice])
    for i in range(num_features):
        if np.random.rand() < harmony_rate:  # Adjust this probability based on your problem
            new_harmony[i] = 1 - new_harmony[i]
    current_obj = objective_function(harmony_memory[rnd_choice])
    new_obj = objective_function(new_harmony)
    if new_obj > current_obj:
        harmony_memory[rnd_choice] = new_harmony
    
#======================================================================================================================

# Get selected features based on final harmony memory
selected_features = harmony_memory[np.argmax([objective_function(i) for i in harmony_memory])]
print('Feature selected')
if len(selected_features) == 0:
    return float('-inf')  # Penalize subsets with no selected features
    
    
# Define the ANN model to be optimized.
def ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=int(neurons_input), activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(int(num_hidden_layers)):
        model.add(tf.keras.layers.Dense(units=int(neurons_hidden), activation='relu'))

    model.add(tf.keras.layers.Dense(units=1))  # Output layer with a single unit for regression.

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Define the search space for Bayesian optimization.
pbounds = {
        'neurons_input': (10, 50),
        'neurons_hidden': (10, 50),
        'num_hidden_layers': (1, 5),
        'learning_rate': (1e-5, 1e-2),
        'batch_size': (8,32 ),
        'epochs': (5, 20)
}

# Define the function to optimize (minimize RMSE).
def optimize_effort_estimation(neurons_input, neurons_hidden, num_hidden_layers, learning_rate, batch_size, epochs):
    model = ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate)
    
    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=int(epochs), verbose=0)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
X_selected = X[:, selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=optimize_effort_estimation,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )

    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
        
    #build the model
    model = ann_model(best_params['neurons_input'],best_params['neurons_hidden'],best_params['num_hidden_layers'],best_params['learning_rate'])
    # Train the model.
    model.fit(X_train, y_train, epochs=int(best_params['epochs']), batch_size=int(best_params['batch_size']), verbose=0)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)


    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
                
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")               
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")            
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")

|   iter    |  target   | harmon... |    hms    | iterat... |
-------------------------------------------------------------
1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 36ms/step


KeyboardInterrupt: 

In [2]:

# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
desharnais=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\desharnais_dataset.csv",header=None,index_col=0)
columns_desharnais=['TeamExp','ManagerExp','YearEnd','Length','Effort','Transactions','Entities','PointsAdjust','Envergure','PointsNonAjust','Language']
desharnais.set_axis(columns_desharnais,axis='columns',inplace=True)
desharnais.rename_axis("Features", axis=1,inplace=True)
desharnais.rename_axis("Projects", axis=0,inplace=True)
dataset = desharnais

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort']).values  # Features
y = dataset['Effort'].values  # Target (effort)

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.function(reduce_retracing=True)


# Step 3: Feature Selection using Harmony Search
def objective_function(subset):
    selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), subset) if is_selected]
    if len(selected_features) == 0:
        return float('-inf')  # Penalize subsets with no selected features
    
    
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    cross_val_rmse = []
    
    X_selected = X[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    error_list=[]
    
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test = y[train_index], y[val_index]
    
        # Step 3: Build the ANN model.
        model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

        # Step 4: Compile the model.
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Step 5: Train the model.
        model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=0)

        # Step 6: Evaluate the model.
        y_pred = model.predict(X_test)
    
        error = np.mean(np.abs(y_pred - y_test))
        error_list.append(error)
        
    return 1 / (1 + np.mean(error_list))  # Fitness is the inverse of the error (higher is better)
#====================================================================================================================
# Harmony Search Parameters
num_features = X.shape[1]
hms = 20  # Harmony memory size
iterations = 100  # Number of iterations

# Initialize harmony memory
harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

# Harmony Search Algorithm
for _ in range(iterations):
    rnd_choice=np.random.choice(hms)
    new_harmony = np.copy(harmony_memory[rnd_choice])
    for i in range(num_features):
        if np.random.rand() < 0.5:  # Adjust this probability based on your problem
            new_harmony[i] = 1 - new_harmony[i]
    current_obj = objective_function(harmony_memory[rnd_choice])
    new_obj = objective_function(new_harmony)
    if new_obj < current_obj:
        harmony_memory[rnd_choice] = new_harmony

# Get selected features based on final harmony memory
selected_features = harmony_memory[np.argmax([objective_function(i) for i in harmony_memory])]
print(selected_features)
#======================================================================================================================

#if len(selected_features) == 0:
#   return float('-inf')  # Penalize subsets with no selected features
    
    
# Define the ANN model to be optimized.
def ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=int(neurons_input), activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(int(num_hidden_layers)):
        model.add(tf.keras.layers.Dense(units=int(neurons_hidden), activation='relu'))

    model.add(tf.keras.layers.Dense(units=1))  # Output layer with a single unit for regression.

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Define the search space for Bayesian optimization.
pbounds = {
        'neurons_input': (10, 50),
        'neurons_hidden': (10, 50),
        'num_hidden_layers': (1, 5),
        'learning_rate': (1e-5, 1e-2),
        'batch_size': (8,32 ),
        'epochs': (5, 20)
}

# Define the function to optimize (minimize RMSE).
def optimize_effort_estimation(neurons_input, neurons_hidden, num_hidden_layers, learning_rate, batch_size, epochs):
    model = ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate)
    
    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=int(epochs), verbose=0)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
X_selected = X[:, selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=optimize_effort_estimation,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )

    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
        
    #build the model
    model = ann_model(best_params['neurons_input'],best_params['neurons_hidden'],best_params['num_hidden_layers'],best_params['learning_rate'])
    # Train the model.
    model.fit(X_train, y_train, epochs=int(best_params['epochs']), batch_size=int(best_params['batch_size']), verbose=0)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)


    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
                
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")               
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")            
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")

1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 31ms/step


...

| 31        | -4.407e+0 | 8.393     | 17.97     | 0.002077  | 48.91     | 46.63     | 2.262     |
1/1 [==============================] - 0s 39ms/step
| 32        | -4.141e+0 | 13.73     | 19.21     | 0.00385   | 37.51     | 49.93     | 3.103     |
1/1 [==============================] - 0s 44ms/step
| 33        | -4.115e+0 | 8.734     | 14.28     | 0.003586  | 39.55     | 48.19     | 4.452     |
1/1 [==============================] - 0s 45ms/step
| 34        | -4.087e+0 | 30.46     | 20.0      | 0.01      | 10.0      | 41.8      | 5.0       |
1/1 [==============================] - 0s 32ms/step
| 35        | -5.613e+0 | 32.0      | 20.0      | 0.01      | 50.0      | 27.49     | 1.0       |
1/1 [==============================] - 0s 46ms/step
| 36        | -3.801e+0 | 8.0       | 20.0      | 0.01      | 40.6      | 44.2      | 5.0       |
1/1 [==============================] - 0s 32ms/step
| 37        | -5.309e+0 | 8.121     | 18.41     | 0.006355  | 10.25     | 28.22     | 1.574     |
1/

1/1 [==============================] - 0s 45ms/step
| 28        | -7.751e+0 | 22.34     | 5.0       | 0.01      | 31.49     | 30.29     | 5.0       |
1/1 [==============================] - 0s 33ms/step
| 29        | -7.787e+0 | 24.99     | 10.3      | 0.01      | 33.26     | 24.63     | 1.105     |
1/1 [==============================] - 0s 42ms/step
| 30        | -7.377e+0 | 23.53     | 11.02     | 0.005731  | 26.89     | 23.41     | 4.369     |
1/1 [==============================] - 0s 46ms/step
| 31        | -5.779e+0 | 25.92     | 11.28     | 0.01      | 30.64     | 27.31     | 5.0       |
1/1 [==============================] - 0s 42ms/step
| 32        | -5.975e+0 | 25.65     | 16.72     | 0.008523  | 30.36     | 25.86     | 4.024     |
1/1 [==============================] - 0s 47ms/step
| 33        | -5.539e+0 | 29.26     | 11.83     | 0.01      | 29.52     | 25.55     | 5.0       |
1/1 [==============================] - 0s 45ms/step
| 34        | -5.48e+03 | 25.28     | 7.388     

1/1 [==============================] - 0s 33ms/step
| 25        | -5.314e+0 | 32.0      | 20.0      | 1e-05     | 26.54     | 30.93     | 1.0       |
1/1 [==============================] - 0s 46ms/step
| 26        | -3.206e+0 | 29.09     | 10.14     | 0.01      | 25.86     | 31.5      | 5.0       |
1/1 [==============================] - 0s 45ms/step
| 27        | -3.54e+03 | 32.0      | 8.78      | 0.01      | 26.95     | 37.95     | 5.0       |
1/1 [==============================] - 0s 32ms/step
| 28        | -3.102e+0 | 8.0       | 20.0      | 0.01      | 44.57     | 18.93     | 1.0       |
1/1 [==============================] - 0s 46ms/step
| 29        | -3.782e+0 | 13.32     | 20.0      | 0.01      | 46.93     | 16.48     | 5.0       |
1/1 [==============================] - 0s 46ms/step
| 30        | -3.17e+03 | 27.46     | 12.45     | 0.01      | 19.14     | 37.15     | 5.0       |
1/1 [==============================] - 0s 45ms/step
| 31        | -3.445e+0 | 19.82     | 15.2      

1/1 [==============================] - 0s 47ms/step
| 22        | -4.884e+0 | 8.0       | 18.0      | 0.01      | 10.33     | 41.72     | 5.0       |
1/1 [==============================] - 0s 45ms/step
| 23        | -4.749e+0 | 11.46     | 7.735     | 0.01      | 38.71     | 48.1      | 5.0       |
1/1 [==============================] - 0s 46ms/step
| 24        | -4.745e+0 | 8.0       | 9.328     | 0.01      | 17.38     | 43.45     | 5.0       |
1/1 [==============================] - 0s 36ms/step
| 25        | -6.656e+0 | 16.14     | 13.35     | 0.001499  | 13.64     | 44.83     | 2.363     |
1/1 [==============================] - 0s 46ms/step
| 26        | -5.542e+0 | 8.0       | 5.811     | 0.01      | 27.48     | 45.61     | 5.0       |
1/1 [==============================] - 0s 46ms/step
| 27        | -4.688e+0 | 8.0       | 20.0      | 0.01      | 10.0      | 32.22     | 5.0       |
1/1 [==============================] - 0s 46ms/step
| 28        | -4.725e+0 | 8.0       | 10.05     

1/1 [==============================] - 0s 46ms/step
| 19        | -6.361e+0 | 21.97     | 20.0      | 0.01      | 10.0      | 50.0      | 5.0       |
1/1 [==============================] - 0s 45ms/step
| 20        | -5.958e+0 | 8.0       | 20.0      | 0.01      | 23.54     | 50.0      | 5.0       |
1/1 [==============================] - 0s 46ms/step
| 21        | -6.154e+0 | 8.0       | 20.0      | 0.01      | 35.2      | 50.0      | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 22        | -5.983e+0 | 19.08     | 20.0      | 0.01      | 25.2      | 50.0      | 5.0       |
1/1 [==============================] - 0s 46ms/step
| 23        | -6.36e+03 | 32.0      | 20.0      | 0.01      | 22.85     | 50.0      | 5.0       |
1/1 [==============================] - 0s 47ms/step
| 24        | -7.559e+0 | 8.0       | 7.638     | 1e-05     | 30.28     | 50.0      | 5.0       |
1/1 [==============================] - 0s 33ms/step
| 25        | -7.555e+0 | 32.0      | 5.0       